```
Licensed to the Apache Software Foundation (ASF) under one
or more contributor license agreements.  See the NOTICE file
distributed with this work for additional information
regarding copyright ownership.  The ASF licenses this file
to you under the Apache License, Version 2.0 (the
"License"); you may not use this file except in compliance
with the License.  You may obtain a copy of the License at
  http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing,
software distributed under the License is distributed on an
"AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
KIND, either express or implied.  See the License for the
specific language governing permissions and limitations
under the License.
```

In [1]:
import os

import geopandas as gpd
from pyspark.sql import SparkSession

from sedona.spark import *

In [2]:
config = SedonaContext.builder() .\
    config('spark.jars.packages',
           'org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.4.1,'
           'org.datasyslab:geotools-wrapper:1.4.0-28.2'). \
    getOrCreate()

sedona = SedonaContext.create(config)


:: loading settings :: url = jar:file:/Users/nileshgajwani/Desktop/spark/spark-3.4.0-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/nileshgajwani/.ivy2/cache
The jars for the packages stored in: /Users/nileshgajwani/.ivy2/jars
org.apache.sedona#sedona-spark-shaded-3.0_2.12 added as a dependency
org.datasyslab#geotools-wrapper added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ade932f0-a9e8-47af-b559-0d52a6a087e9;1.0
	confs: [default]
	found org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 in central
	found org.datasyslab#geotools-wrapper;1.4.0-28.2 in central
:: resolution report :: resolve 81ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.sedona#sedona-spark-shaded-3.0_2.12;1.4.1 from central in [default]
	org.datasyslab#geotools-wrapper;1.4.0-28.2 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------

## Geometry Constructors

### ST_Point

In [3]:
point_csv_df = sedona.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").\
    load("data/testpoint.csv")

point_csv_df.createOrReplaceTempView("pointtable")

point_df = sedona.sql("select ST_Point(cast(pointtable._c0 as Decimal(24,20)), cast(pointtable._c1 as Decimal(24,20))) as arealandmark from pointtable")
point_df.show(5)

+-----------------+
|     arealandmark|
+-----------------+
|POINT (1.1 101.1)|
|POINT (2.1 102.1)|
|POINT (3.1 103.1)|
|POINT (4.1 104.1)|
|POINT (5.1 105.1)|
+-----------------+
only showing top 5 rows



### ST_GeomFromText

In [4]:
polygon_wkt_df = sedona.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/county_small.tsv")

polygon_wkt_df.createOrReplaceTempView("polygontable")
polygon_df = sedona.sql("select polygontable._c6 as name, ST_GeomFromText(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+----------------+--------------------+
|            name|         countyshape|
+----------------+--------------------+
|   Cuming County|POLYGON ((-97.019...|
|Wahkiakum County|POLYGON ((-123.43...|
|  De Baca County|POLYGON ((-104.56...|
|Lancaster County|POLYGON ((-96.910...|
| Nuckolls County|POLYGON ((-98.273...|
+----------------+--------------------+
only showing top 5 rows



### ST_GeomFromWKB

In [5]:
polygon_wkb_df = sedona.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/county_small_wkb.tsv")

polygon_wkb_df.createOrReplaceTempView("polygontable")
polygon_df = sedona.sql("select polygontable._c6 as name, ST_GeomFromWKB(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+----------------+--------------------+
|            name|         countyshape|
+----------------+--------------------+
|   Cuming County|POLYGON ((-97.019...|
|Wahkiakum County|POLYGON ((-123.43...|
|  De Baca County|POLYGON ((-104.56...|
|Lancaster County|POLYGON ((-96.910...|
| Nuckolls County|POLYGON ((-98.273...|
+----------------+--------------------+
only showing top 5 rows



### ST_GeomFromGeoJSON

In [6]:
polygon_json_df = sedona.read.format("csv").\
    option("delimiter", "\t").\
    option("header", "false").\
    load("data/testPolygon.json")

polygon_json_df.createOrReplaceTempView("polygontable")
polygon_df = sedona.sql("select ST_GeomFromGeoJSON(polygontable._c0) as countyshape from polygontable")
polygon_df.show(5)

+--------------------+
|         countyshape|
+--------------------+
|POLYGON ((-87.621...|
|POLYGON ((-85.719...|
|POLYGON ((-86.000...|
|POLYGON ((-86.574...|
|POLYGON ((-85.382...|
+--------------------+
only showing top 5 rows



## Spatial Operations

### Spatial Join - Distance Join

In [7]:
point_csv_df_1 = sedona.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").load("data/testpoint.csv")

point_csv_df_1.createOrReplaceTempView("pointtable")

point_df1 = sedona.sql("SELECT ST_Point(cast(pointtable._c0 as Decimal(24,20)),cast(pointtable._c1 as Decimal(24,20))) as pointshape1, \'abc\' as name1 from pointtable")
point_df1.createOrReplaceTempView("pointdf1")

point_csv_df2 = sedona.read.format("csv").\
    option("delimiter", ",").\
    option("header", "false").load("data/testpoint.csv")

point_csv_df2.createOrReplaceTempView("pointtable")
point_df2 = sedona.sql("select ST_Point(cast(pointtable._c0 as Decimal(24,20)),cast(pointtable._c1 as Decimal(24,20))) as pointshape2, \'def\' as name2 from pointtable")
point_df2.createOrReplaceTempView("pointdf2")

distance_join_df = sedona.sql("select * from pointdf1, pointdf2 where ST_Distance(pointdf1.pointshape1,pointdf2.pointshape2) < 2")
distance_join_df.explain()
distance_join_df.show(5)

== Physical Plan ==
BroadcastIndexJoin pointshape2#253: geometry, LeftSide, LeftSide, Inner, INTERSECTS, ( **org.apache.spark.sql.sedona_sql.expressions.ST_Distance**   < 2.0) ST_INTERSECTS(pointshape1#228, pointshape2#253)
:- SpatialIndex pointshape1#228: geometry, QUADTREE, false, 2.0
:  +- Project [ **org.apache.spark.sql.sedona_sql.expressions.ST_Point**   AS pointshape1#228, abc AS name1#229]
:     +- FileScan csv [_c0#224,_c1#225] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/nileshgajwani/Desktop/sedona/sedona/binder/data/testpoint...., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<_c0:string,_c1:string>
+- Project [ **org.apache.spark.sql.sedona_sql.expressions.ST_Point**   AS pointshape2#253, def AS name2#254]
   +- FileScan csv [_c0#249,_c1#250] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/nileshgajwani/Desktop/sedona/sedona/binder/data/testpoint...., PartitionFilt

### Spatial Join - Range Join and RDD API Join

Please refer to the example - airports per country: https://github.com/apache/sedona/blob/master/binder/ApacheSedonaSQL_SpatialJoin_AirportsPerCountry.ipynb

### Converting GeoPandas to Apache Sedona

In [8]:
import pandas as pd
gdf = gpd.read_file("data/gis_osm_pois_free_1.shp")
gdf = gdf.replace(pd.NA, '')
osm_points = sedona.createDataFrame(
    gdf
)

In [9]:
osm_points.printSchema()

root
 |-- osm_id: string (nullable = true)
 |-- code: long (nullable = true)
 |-- fclass: string (nullable = true)
 |-- name: string (nullable = true)
 |-- geometry: geometry (nullable = true)



In [10]:
osm_points.show(5)

+--------+----+---------+--------------+--------------------+
|  osm_id|code|   fclass|          name|            geometry|
+--------+----+---------+--------------+--------------------+
|26860257|2422|camp_site|      de Kroon|POINT (15.3393145...|
|26860294|2406|   chalet|Leśne Ustronie|POINT (14.8709625...|
|29947493|2402|    motel|              |POINT (15.0946636...|
|29947498|2602|      atm|              |POINT (15.0732014...|
|29947499|2401|    hotel|              |POINT (15.0696777...|
+--------+----+---------+--------------+--------------------+
only showing top 5 rows



In [11]:
osm_points.createOrReplaceTempView("points")

In [12]:
transformed_df = sedona.sql(
    """
        SELECT osm_id,
               code,
               fclass,
               name,
               ST_Transform(geometry, 'epsg:4326', 'epsg:2180') as geom 
        FROM points
    """)

In [13]:
transformed_df.show(5)

+--------+----+---------+--------------+--------------------+
|  osm_id|code|   fclass|          name|                geom|
+--------+----+---------+--------------+--------------------+
|26860257|2422|camp_site|      de Kroon|POINT (-3288183.3...|
|26860294|2406|   chalet|Leśne Ustronie|POINT (-3341183.9...|
|29947493|2402|    motel|              |POINT (-3320466.5...|
|29947498|2602|      atm|              |POINT (-3323205.7...|
|29947499|2401|    hotel|              |POINT (-3323655.1...|
+--------+----+---------+--------------+--------------------+
only showing top 5 rows



In [14]:
transformed_df.createOrReplaceTempView("points_2180")

In [15]:
neighbours_within_1000m = sedona.sql("""
        SELECT a.osm_id AS id_1,
               b.osm_id AS id_2,
               a.geom 
        FROM points_2180 AS a, points_2180 AS b 
        WHERE ST_Distance(a.geom,b.geom) < 50
    """)

In [16]:
neighbours_within_1000m.show()

23/07/03 21:13:53 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


+----------+---------+--------------------+
|      id_1|     id_2|                geom|
+----------+---------+--------------------+
| 197624402|197624402|POINT (-3383818.5...|
| 197663196|197663196|POINT (-3383367.1...|
| 197953474|197953474|POINT (-3383763.3...|
| 262310516|262310516|POINT (-3384257.6...|
|1074233123|262310516|POINT (-3384262.1...|
| 270281140|270281140|POINT (-3385421.2...|
|1074232906|270281140|POINT (-3385408.6...|
| 270306609|270306609|POINT (-3383982.8...|
| 270306746|270306746|POINT (-3383898.4...|
| 280402616|280402616|POINT (-3378817.6...|
| 839725400|280402616|POINT (-3378841.1...|
| 293896571|293896571|POINT (-3385029.0...|
|3256728465|293896571|POINT (-3385002.4...|
| 310838954|310838954|POINT (-3390510.5...|
| 311395303|311395303|POINT (-3389444.4...|
| 311395425|311395425|POINT (-3389867.6...|
|6339786017|311395425|POINT (-3389850.1...|
| 825853330|311395425|POINT (-3389877.4...|
| 945009922|311395425|POINT (-3389878.6...|
| 320100848|320100848|POINT (-33

## Converting Apache Sedona to GeoPandas

In [17]:
df = neighbours_within_1000m.toPandas()

23/07/03 21:13:54 WARN JoinQuery: UseIndex is true, but no index exists. Will build index on the fly.


In [18]:
gdf = gpd.GeoDataFrame(df, geometry="geom")

In [19]:
gdf

,id_1,id_2,geom
0,197624402,197624402,POINT (-3383818.580 4179182.169)
1,197663196,197663196,POINT (-3383367.151 4179427.096)
2,197953474,197953474,POINT (-3383763.332 4179408.785)
3,262310516,262310516,POINT (-3384257.682 4178033.053)
4,1074233123,262310516,POINT (-3384262.187 4178036.442)
...,...,...,...
45314,6785548354,6785548354,POINT (-3271487.870 4337964.529)
45315,6785548356,6785548356,POINT (-3273379.389 4338379.126)
45316,6785548357,6785548357,POINT (-3273745.222 4338528.241)
45317,6785548358,6785548358,POINT (-3273027.996 4338093.401)
